In [1]:
import os
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd
from easydict import EasyDict

sys.path.append("../../../..")
from data.classification.toxic_comments.load import ToxicComtDataLoader
from src.graphs.models.toxic_comment import ToxicComtModel
from src.graphs.models.word_embedding import WordEmbedding

In [2]:
CONFIG = EasyDict({
    "model": {
        "comment_encoder": {
            "input_size": 50,
            "hidden_size": 32,
            "num_layers": 3
        },
        "output_layer": {
            "n_factor": 32,
            "n_target": 6
        }
    },
    "embedding": {
        "path_dir": "../../../../data/word_embedding/glove",
        "path_file": "glove.6B.50d.txt",
        "num_embeddings": 0,
        "embedding_dim": 50
    },
    "data": {
        "path_dir": "../../../../data/classification/toxic_comments",
        "path_file": "sample.csv",
        "pct_train": 0.7,
        "batch_size": 4,
        "min_freq": 4
    },
    "train": {
        "lr": 0.01
    }
})

In [3]:
dataloader = ToxicComtDataLoader(CONFIG.data)

In [4]:
batch_x, batch_y = next(iter(dataloader.loader_train))

In [5]:
vocab = dataloader.vocab
vocab.load_vectors(CONFIG.embedding.path_file, CONFIG.embedding.path_dir)


In [6]:
temp_embedding = WordEmbedding(CONFIG.embedding)
temp_embedding.from_pretrained(vocab.vocab)

In [7]:
model = ToxicComtModel(CONFIG.model)

In [8]:
model(temp_embedding(batch_x)) - batch_y

tensor([[[0.5490, 0.4890, 0.4564, 0.4723, 0.5040, 0.5266],
         [0.5490, 0.4890, 0.4564, 0.4723, 0.5040, 0.5266],
         [0.5477, 0.4797, 0.4878, 0.4650, 0.4862, 0.5295],
         [0.5490, 0.4890, 0.4564, 0.4723, 0.5040, 0.5266]],

        [[0.5704, 0.4562, 0.4919, 0.4703, 0.4580, 0.5788],
         [0.5704, 0.4562, 0.4919, 0.4703, 0.4580, 0.5788],
         [0.5761, 0.4581, 0.5066, 0.4952, 0.4480, 0.5555],
         [0.5704, 0.4562, 0.4919, 0.4703, 0.4580, 0.5788]],

        [[0.5351, 0.5203, 0.4247, 0.4617, 0.5108, 0.4931],
         [0.5351, 0.5203, 0.4247, 0.4617, 0.5108, 0.4931],
         [0.5425, 0.4984, 0.4133, 0.4459, 0.5208, 0.4764],
         [0.5351, 0.5203, 0.4247, 0.4617, 0.5108, 0.4931]]],
       grad_fn=<SubBackward0>)